In [1]:
import pandas as pd
import numpy as np

In [3]:
# 1. Load the dataset
# We use na_values=['?'] because this specific dataset uses '?' for missing info
df = pd.read_csv("D:\Domain\Infosys Springboard Internship\Optimization_of_IT_Support_Team_Performance_using_Advanced_Analytics\Data\Raw Data\incident_event_log.csv", na_values=['?', ''])

<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
C:\Users\manit\AppData\Local\Temp\ipykernel_61588\2611436767.py:3: SyntaxWarning: invalid escape sequence '\D'
  df = pd.read_csv("D:\Domain\Infosys Springboard Internship\Optimization_of_IT_Support_Team_Performance_using_Advanced_Analytics\Data\Raw Data\incident_event_log.csv", na_values=['?', ''])
C:\Users\manit\AppData\Local\Temp\ipykernel_61588\2611436767.py:3: DtypeWarning: Columns (19,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("D:\Domain\Infosys Springboard Internship\Optimization_of_IT_Support_Team_Performance_using_Advanced_Analytics\Data\Raw Data\incident_event_log.csv", na_values=['?', ''])


In [4]:
print("Raw Data Shape:", df.shape)
print("Columns:", df.columns)
df.head()

Raw Data Shape: (141712, 36)
Columns: Index(['number', 'incident_state', 'active', 'reassignment_count',
       'reopen_count', 'sys_mod_count', 'made_sla', 'caller_id', 'opened_by',
       'opened_at', 'sys_created_by', 'sys_created_at', 'sys_updated_by',
       'sys_updated_at', 'contact_type', 'location', 'category', 'subcategory',
       'u_symptom', 'cmdb_ci', 'impact', 'urgency', 'priority',
       'assignment_group', 'assigned_to', 'knowledge',
       'u_priority_confirmation', 'notify', 'problem_id', 'rfc', 'vendor',
       'caused_by', 'closed_code', 'resolved_by', 'resolved_at', 'closed_at'],
      dtype='object')


,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [5]:
df.drop_duplicates(inplace=True)
print("After duplicate removal:", df.shape)

After duplicate removal: (141712, 36)


In [6]:
# Convert date columns to datetime objects
date_cols = [
    "opened_at",
    "resolved_at",
    "closed_at",
    "sys_created_at",
    "sys_updated_at"
]

for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)


In [7]:
print(df[date_cols].dtypes)

opened_at         datetime64[ns]
resolved_at       datetime64[ns]
closed_at         datetime64[ns]
sys_created_at    datetime64[ns]
sys_updated_at    datetime64[ns]
dtype: object


In [16]:
df.head()
df.columns.tolist()

['number',
 'incident_state',
 'active',
 'reassignment_count',
 'reopen_count',
 'sys_mod_count',
 'made_sla',
 'caller_id',
 'opened_by',
 'opened_at',
 'sys_created_by',
 'sys_created_at',
 'sys_updated_by',
 'sys_updated_at',
 'contact_type',
 'location',
 'category',
 'subcategory',
 'u_symptom',
 'cmdb_ci',
 'impact',
 'urgency',
 'priority',
 'assignment_group',
 'assigned_to',
 'knowledge',
 'u_priority_confirmation',
 'notify',
 'problem_id',
 'rfc',
 'vendor',
 'caused_by',
 'closed_code',
 'resolved_by',
 'resolved_at',
 'closed_at']

In [9]:
df.sort_values(
    by=["number", "sys_updated_at"],
    inplace=True
)

df.reset_index(drop=True, inplace=True)
print("After sorting:", df.shape)

After sorting: (141712, 36)


In [10]:
bool_cols = ["active", "made_sla", "u_priority_confirmation"]

for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].astype("bool")
print(df[bool_cols].dtypes)

active                     bool
made_sla                   bool
u_priority_confirmation    bool
dtype: object


In [11]:
# Define the rules for flattening
agg_rules = {
    'opened_at': 'first',           # Start time = The first time we saw it
    'resolved_at': 'last',          # End time = The last update
    'closed_at': 'last',            # Backup End time
    'category': 'last',             # Final category it ended up in
    'subcategory': 'last',          # Final subcategory
    'priority': 'last',             # Final priority status
    'assignment_group': 'last',     # Who had it last
    'incident_state': 'last',       # Did it end up Closed or Resolved?
    'reassignment_count': 'max',    # Max times it was passed around
    'reopen_count': 'max'           # Max times it was reopened
}

df_clean = df.groupby('number').agg(agg_rules).reset_index()
print(f"Data compressed from {len(df)} rows to {len(df_clean)} unique tickets.")

Data compressed from 141712 rows to 24918 unique tickets.


In [12]:
# Create 'Resolution_Time_Hours'
# Logic: Use 'resolved_at'. If that's missing, use 'closed_at'.
df_clean['final_end_time'] = df_clean['resolved_at'].fillna(df_clean['closed_at'])

# Calculate the difference in hours
df_clean['Resolution_Time_Hours'] = (df_clean['final_end_time'] - df_clean['opened_at']).dt.total_seconds() / 3600

In [13]:
# tickets with negative time or 0 time (data errors)
df_clean = df_clean[df_clean['Resolution_Time_Hours'] > 0]

In [14]:
df_clean[['number', 'opened_at', 'final_end_time', 'Resolution_Time_Hours']].head()

,number,opened_at,final_end_time,Resolution_Time_Hours
0,INC0000045,2016-02-29 01:16:00,2016-02-29 11:29:00,10.216667
1,INC0000047,2016-02-29 04:40:00,2016-03-01 09:52:00,29.200000
2,INC0000057,2016-02-29 06:10:00,2016-03-01 02:55:00,20.750000
3,INC0000060,2016-02-29 06:38:00,2016-03-02 12:06:00,53.466667
4,INC0000062,2016-02-29 06:58:00,2016-02-29 15:51:00,8.883333


In [17]:
# Create numeric priority score
priority_map = {
    '1 - Critical': 4,
    '2 - High': 3,
    '3 - Moderate': 2,
    '4 - Low': 1
}

if 'priority' in df_clean.columns:
    df_clean['priority_score'] = df_clean['priority'].map(priority_map)

In [18]:
df.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,2016-02-29 11:29:00,2016-03-05 12:00:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,2016-02-29 11:29:00,2016-03-05 12:00:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,2016-02-29 11:29:00,2016-03-05 12:00:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,2016-02-29 11:29:00,2016-03-05 12:00:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,2016-02-29 04:40:00,...,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 81,2016-03-01 09:52:00,2016-03-06 10:00:00


In [19]:
# Create resolution speed buckets
df_clean['resolution_category'] = pd.cut(
    df_clean['Resolution_Time_Hours'],
    bins=[0, 24, 72, 168, 99999],
    labels=['Fast (<1 day)', 'Normal (1-3 days)', 'Slow (3-7 days)', 'Very Slow (>7 days)']
)

In [20]:
df_clean['opened_year'] = df_clean['opened_at'].dt.year
df_clean['opened_month'] = df_clean['opened_at'].dt.month
df_clean['opened_dayofweek'] = df_clean['opened_at'].dt.day_name()

In [21]:
df_clean['is_resolved'] = df_clean['final_end_time'].notna().astype(int)

In [23]:
df_clean.to_csv("powerbi_ready_data.csv", index=False)
print("Power BI dataset saved successfully.")

Power BI dataset saved successfully.
